# U of M Lab Group Management Tools

Load Required Packages

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import pprint
import yaml
import pandas as pd
from uuid import uuid4
from glob import glob
pp = pprint.PrettyPrinter()

# Create Canvas Groups

This first tool uses the Canvas API to randomly assign all students lab groups. The dynamic input includes <strong>a token, the canvas course IDs, students per group, lab titles and lab numbers.</strong>
* <strong>Tokens</strong> can be created by a user by logging into canvas and going to Account -> Settings -> '+New Access Token'. This is hard coded as the variable `token`
* The canvas <strong>course IDs</strong> are identical to the ones used by canvas on the browser, e.g. going to Physics 141 WN 2020 canvas page the url is `https://umich.instructure.com/courses/358422` and the digits at the end are the course ID, `358422` in this case. They are hard coded as the variable `course_id`
* When a <strong>lab number</strong> is specified, the set of groups for that lab is the one which will be created. Running the script will only produce one set of groups, in principle the LGSI should run this weekly as the rosters will continually update throughout the semester. Hard coded in the as `lab_number`
* The <strong>lab titles</strong> are held in a simple, hard coded array `lab_names` and will show up as the title of the set of canvas groups for a given lab. When $n=$`lab_number` is specified the $n$th element of the `lab_names` will be selected as the title for the groups being processed.

The random group generated was written by Nathaniel Lydick and edited by Eric Gonzalez.
The zoom group addition was written by Eric Gonzalez.

In [15]:
token = '1770~beRDsqExEBG43YaBTqzcgo7KMIlBepPjqNr8iRLZM2xvUQJKdyYxWWyuFHrVflx9'

lab_number = 8
phys_course = 241

########################################################################

if phys_course == 161:
    students_per_group = 4
    course_id = 407048
    lab_names = [ 
            'Tutorial',
            'Introduction to Statistics and Data Analysis',
            'Motion with Uniform Acceleration',
            'Motion with Non-Uniform Acceleration',
            'Simple Harmonic Motion',
            'The Physical Pendulum',
            'Waves and Sound',
            'One Dimensional Collisions',
            'Two Dimensional Collisions',
            'The Rotating Bar',
            'The Gyroscopoe']

elif phys_course == 141:
    students_per_group = 4
    course_id = 436091
    lab_names = [ 
        'Tutorial',
        'Introduction to Statistics and Data Analysis',
        'Computation of Kinematics',
        'Motion Due to Uniform Acceleration',
        'Free Fall and Air Drag',
        'Two Dimensional Motion with Drag',
        'One Dimensional Collisions',
        'Two Dimensional Collisiions',
        'The Inclined Plane',
        'The Gyroscope',
        'Gravitational Motion']
    
elif phys_course == 241:
    students_per_group = 4
    course_id = 436130
    lab_names = ['Tutorial',
            'Electric Fields',
            'Electric Fields Computational Lab',
            'Capacitance',
            'DC Circuits',
            'Magnetic Fields and Magnetic Forces',
            'Motion of Charged Particles in Fields',
            'Charged Particles Computational Lab',
            'Faraday\'s Law',
            'AC Circuits',
            'AC Circuits Computational Lab']

elif phys_course == 261:
    students_per_group = 4
    course_id = 407049
    lab_names = ['Tutorial',
            'Electric Fields',
            'Electric Fields Computational Lab',
            'Capacitance',
            'DC Circuits',
            'Magnetic Fields and Magnetic Forces',
            'Motion of Charged Particles in Fields',
            'Charged Particles Computational Lab',
            'Faraday\'s Law',
            'AC Circuits',
            'AC Circuits Computational Lab']

zb_path = 'Physics {}/Zoom Groups/'.format(phys_course)

Define functions to use Canvas API

In [16]:

# set the group_set_id to an integer to use that group set rather than making a new one
# set the group_set_id to None to create a new group set with the specified name (below)
# NOTE: This will not delete existing groups from the group set, though it WILL re-assign the students in them
#       ^ that behavior could be added (see delete(...) below), but it seems to me it's easier to just make new groups
# group_set_id = 21746
group_set_id = None


full_lab_name = 'Lab {:d}: {}'.format(lab_number, lab_names[lab_number])
group_set_name = full_lab_name

group_name_fmt = 'Lab {} {} - Group {}'

#########################################################################################
# Obviously you can edit the stuff below, but I tried to make it so you shouldn't have to
#########################################################################################

import requests
import sys
import math
# from pprint import pprint as print
import random
# random number generator for shuffling the groups
# r = random.SystemRandom()
r = random.Random() # we'll use the python random instead so we can seed it

def format_url(url, v1=True, rawURL=False):
    if rawURL:
        return url

    # assumed v1 specified by url
    if '/api/' in url:
        #return 'https://umich.test.instructure.com' + url
        return 'https://umich.instructure.com' + url
    if 'api/' in url:
        #return 'https://umich.test.instructure.com/' + url
        return 'https://umich.instructure.com/' + url

    #canvasURL = 'https://umich.test.instructure.com/api/lti/'
    canvasURL = 'https://umich.instructure.com/api/lti/' # resets weekly, use .test. for monthly

    if v1:
        #canvasURL = 'https://umich.test.instructure.com/api/v1/'
        canvasURL = 'https://umich.instructure.com/api/v1/' # resets weekly

        # print('Raw url: ' + url)
    return canvasURL + url

def get(url, params=None, all=True, v1 = True, rawURL=False):
    canvasURL = format_url(url,v1,rawURL)
    response = {}

    if 'per_page' not in url:
        default_page_amount = 25
        if '?' in url:
            canvasURL += '&per_page=%d' % default_page_amount
        else:
            canvasURL += '?per_page=%d' % default_page_amount

    try:
        got = requests.get(canvasURL, params=params, headers={'Authorization': 'Bearer %s'%token})
        response = got.json()
        if 'errors' not in response:
            if all and 'next' in got.links:
                # print('Continuation: ' + str(got.links['next']))
                # print(got.links['next']['url'])
                # print(str(got.links))
                next = got.links['next']['url']
                response += get(next,params,all=True,v1=v1,rawURL=True)
            return response
    except Exception as e:
        print("Exceptions thrown in GET: " + str(e),file=sys.stderr)
        raise e
    print("Could not GET url: " + canvasURL,file=sys.stderr)
    if 'errors' in response:
        print("Error:",file=sys.stderr)
        print(response['errors'],file=sys.stderr)
    return response


def post(url, dictData=None, v1 = True, retry=5):
    canvasURL = format_url(url,v1,False)
    response = {}
    for attempt in range(1+int(retry)):
        if attempt > 0:
            print('Retrying [{}] POST {}'.format(attempt,canvasURL))
        try:
            req = requests.post(canvasURL,data=dictData, headers={'Authorization': 'Bearer %s'%token})
            response.update(req.json())
            if 'errors' not in response:
                return response
        except Exception as e:
            print("Exceptions thrown in POST: " + str(e),file=sys.stderr)
            response['python_exception_message'] = str(e)
            response['python_exception'] = e
        print("Could not POST url: " + canvasURL,file=sys.stderr)
        if 'errors' in response:
            print("Error:",file=sys.stderr)
            print(response['errors'],file=sys.stderr)
        else:
            response['errors'] = 'PYTHON-EXCEPTION'
    return response

def put(url, dictData=None, v1 = True, retry=5):
    canvasURL = format_url(url,v1,False)
    response = {}
    for attempt in range(1+int(retry)):
        if attempt > 0:
            print('Retrying [{}] PUT {}'.format(attempt,canvasURL))
        try:
            req = requests.put(canvasURL,data=dictData, headers={'Authorization': 'Bearer %s'%token})
            response.update(req.json())
            if 'errors' not in response:
                return response
        except Exception as e:
            print("Exceptions thrown in PUT: " + str(e),file=sys.stderr)
            response['python_exception_message'] = str(e)
            response['python_exception'] = e
        print("Could not PUT url: " + canvasURL,file=sys.stderr)
        if 'errors' in response:
            print("Error:",file=sys.stderr)
            print(response['errors'],file=sys.stderr)
        else:
            response['errors'] = 'PYTHON-EXCEPTION'

    return response

def delete(url, v1 = True, retry=5):
    canvasURL = format_url(url,v1,False)
    response = {}
    for attempt in range(1+int(retry)):
        if attempt > 0:
            print('Retrying [{}] DELETE {}'.format(attempt,canvasURL))
        try:
            req = requests.delete(canvasURL, headers={'Authorization': 'Bearer %s'%token})
            response.update(req.json())
            if 'errors' not in response:
                return response
        except Exception as e:
            print("Exceptions thrown in DELETE: " + str(e),file=sys.stderr)
        print("Could not DELETE url: " + canvasURL,sys.stderr)
        if 'errors' in response:
            print("Error:",file=sys.stderr)
            print(response['errors'],file=sys.stderr)
    return response

delete = lambda : print('DELETE has been disabled to prevent potential removal of unintended groups',sys.stderr)

def make_group_set(course_id, name, **kwargs):
    # the group set must not have a taken name, or there will be an error (400?) posting the group. The PUT api must be used in that case.
    # create_group_count=None, self_signup=None, auto_leader=None, group_limit=None
    defaults = {
        'name' : name,
        'create_group_count':None,
        'self_signup':None,
        'auto_leader':None,
        'group_limit':None
    }
    defaults.update(kwargs)
    return post('/api/v1/courses/{course_id}/group_categories'.format(course_id=course_id),
         dictData=defaults)

def delete_group_set(group_id):
    return delete('/api/v1/group_categories/{}'.format(group_id))


def get_group_sets(courseID):
    return get('courses/{course_id}/group_categories'.format(course_id=courseID))

def get_group_set_groups(groupSetID):
    return get('group_categories/{}/groups'.format(groupSetID))


def get_sections(course_id, include=['students']):
    return get('courses/{}/sections'.format(course_id),params=({'include':include} if include is not None else None))


def get_section_names(course_id):
    return list(map(lambda x : x['name'],get_sections(course_id,include=None)))

def get_section_students(section_id):
    return get('sections/{}'.format(section_id),params={'include':'students'})

def get_students_by_section(course_id):
    sects = get_sections(course_id,include=['students'])
    out = {}
    for sect in sects:
        l = []
        for s in sect['students']:
            l.append(s['id'])
        out[sect['id']] = l
    return out

def make_group(group_set_id, name, **kwargs):
    defaults = {
      'name': name,
      'description': None,
      'is_public': False,
      'join_level': 'invitation_only',
    }
    defaults.update(kwargs)
    url = 'group_categories/{}/groups'.format(group_set_id)
    # if group_set_id is None:
    #     url = 'groups'
    return post(url,dictData=defaults)

def set_members_for_group(group_id, user_ids):
    # for user_id in user_ids:
    #     post('groups/{}/memberships'.format(group_id),dictData={'user_id':str(user_id)})
    # return None
    id_str = user_ids
    return put('groups/{}'.format(group_id),dictData={'members[]':id_str})

    # return requests.put('http://httpbin.org/anything',data={'members[]':id_str}).json()

def form_groups_in_group_set(course_id, group_set_id):
    sections = get_sections(course_id)
    trim_section_name = lambda x : x[x.find('Section'):] if 'Section' in x else x
    section_names = dict(map(lambda x : (x['id'],trim_section_name(x['name'])),sections))

    total_group_count = 0

    section_student_ids = {}
    for sect in sections:
        l = []
        try:
            for s in sect['students']:
                l.append(s['id'])
            section_student_ids[sect['id']] = l
        except:
            print('exception on:\n')
            print(sect)

    for section_id in section_names:
        try:
            current_students = section_student_ids[section_id]
            r.seed(a=3**lab_number+lab_number+256*section_id)
            r.shuffle(current_students)

            # I was going to go for the fixed group count. Instead, I'll just evenly
            #    distribute the pad slots from making the last group
            # how many students to pad in the groups
            # pad_count = group_count*students_per_group - len(current_students)

            if len(current_students) > 20: students_per_group = 5
            else: students_per_group = 4

            ending_group_size = len(current_students) % students_per_group
            pad_students = 0 if ending_group_size == 0 else (students_per_group - ending_group_size)
            group_count = math.ceil(len(current_students) / students_per_group)

            pad_groups = r.sample(range(group_count),pad_students)


            for i in range(group_count):
                group_size = students_per_group
                # This randomly assigns the smaller sized groups
                if i in pad_groups:
                    group_size -= 1
                # If we want them at the beginning, I think `if i < pad_students:` would work
                group = current_students[:group_size]
                current_students = current_students[group_size:]
                group_name = group_name_fmt.format(lab_number, section_names[section_id], i+1)
                g = make_group(group_set_id,group_name)
                gid = g['id']
                added = set_members_for_group(gid, group)
            print("Added section {}'s {} groups".format(section_names[section_id],group_count))
            total_group_count += group_count
        except: None

    return total_group_count


Use Canvas API to create groups

In [17]:

import pandas as pd

#def zoom_breakout_rooms(section,group_set)
    

# Code to find the course by name rather than using a given id
# json = get('courses')
# print(json)
# print( [x['course_code'] for x in json])
# ids = [x['id'] for x in json if '136' in x['course_code']]
# print(ids)
# i = ids[0]
# course_id = i

if course_id is None:
    print('We must have a course id to add the group set to it.')
    print('The course id can be obtained from the URL for the course page on Canvas')
    print('  https://umich.beta.instructure.com/courses/<COURSE ID>')
    exit()

groupsets = get_group_sets(course_id)
gs_names = list(map(lambda x : x['name'], groupsets))
if group_set_id is None:

    if group_set_name in gs_names:
        conflict_id = ', '.join([str(x['id']) for x in groupsets if group_set_name == x['name']])
        print('We cannot create the group set with name {}.'.format(group_set_name))
        print('The group already exists with id={}'.format(conflict_id))
        print('Specify the group_set_id in the script to avoid this error, or choose a different group_set_name')
        exit()

    print('Making Group Set: ' + group_set_name)
    group_set = make_group_set(course_id, group_set_name)

    group_set_id = group_set['id']
    print('Created Group Set: %d'%group_set_id)

# if we wanted to delete the group that existed (but this would be much better done through canvas api
# gs = list(map(lambda x : (x['id'],x['name']),groupsets))
# for i,name in gs:
#     if name == 'Testing automated groups - 2':
#         delete_group_set(i)

print('Forming Groups')
count = form_groups_in_group_set(course_id, group_set_id)
print('Finished adding {} groups to Group Set "{}" [{}]'.format(count,group_set_name,group_set_id))
print("It's url might(?) be: https://umich.beta.instructure.com/courses/{}/groups#tab-{}".format(course_id,group_set_id))
#exit()




Making Group Set: Lab 8: Faraday's Law
Created Group Set: 32367
Forming Groups


Added section Section 003's 5 groups


Added section Section 010's 5 groups


Added section Section 011's 5 groups


Added section Section 012's 5 groups


...

Added section Section 070's 5 groups


Added section Section 071's 5 groups


Added section Section 072's 5 groups


Added section Section 073's 5 groups


Added section Section 074's 5 groups
Finished adding 110 groups to Group Set "Lab 8: Faraday's Law" [32367]
It's url might(?) be: https://umich.beta.instructure.com/courses/436130/groups#tab-32367


Retrieve created groups to write to csv for Zoom breakout rooms

In [18]:

import pandas as pd
import os

#zb_path 
groupsets = get_group_sets(course_id)

for group in groupsets:
    try:        
        if int(group['name'][4]) == lab_number:
            group_ids = get_group_set_groups(group['id'])
            print('Recovered group ids for output to zoom .csv')
        elif int(group['name'][4:6]) == lab_number:
            group_ids = get_group_set_groups(group['id'])
            print('Recovered group ids for output to zoom .csv')
    except: None
        
df = pd.DataFrame(data=None,columns=['Pre-assign Room Name','Email Address'])

#if phys_course == 261: cap_num = 4
#else: cap_num = 5
cap_num = 5
    
for grp in group_ids:
    this_name = grp['name']
    sec = this_name.find('Section')+8
    grp_num = this_name.find('Group')+6
    section = this_name[sec:sec+3]
    group_num = this_name[grp_num]
    
    if phys_course == 261 and section == '001': cap_num = 4
    elif phys_course == 261 and section == '003': cap_num = 2

    else: cap_num = 5
    
    for student in get('groups/{}/users'.format(grp['id'])):
        zs = student['login_id']+'@umich.edu'
        zb_series = {'Pre-assign Room Name': this_name, 'Email Address':zs}
        df = df.append(zb_series,ignore_index=True)
    if int(group_num) == cap_num:
        if not os.path.exists(zb_path):
            os.mkdir(zb_path)
        final_path = '{}Lab {}/'.format(zb_path,lab_number)
        if not os.path.exists(final_path):
            os.mkdir(final_path)
        print('Output {} groups to section {} lab {}'.format(group_num,section,lab_number))
        fname_zb_csv = '{}Section {} Lab {}.csv'.format(final_path,section,lab_number)
        df.to_csv(fname_zb_csv,index=False)
        df = pd.DataFrame(data=None,columns=['Pre-assign Room Name','Email Address'])
print('Done.')


Recovered group ids for output to zoom .csv


Output 5 groups to section 003 lab 8


Output 5 groups to section 010 lab 8


Output 5 groups to section 011 lab 8


Output 5 groups to section 012 lab 8


...

Output 5 groups to section 070 lab 8


Output 5 groups to section 071 lab 8


Output 5 groups to section 072 lab 8


Output 5 groups to section 073 lab 8


Output 5 groups to section 074 lab 8
Done.


In [6]:
def update_zoom_groups_by_section(input_section,lab_number):
    groupsets = get_group_sets(course_id)

    for group in groupsets:
        try:        
            if int(group['name'][4]) == lab_number:
                group_ids = get_group_set_groups(group['id'])
                print('Recovered group ids for output to zoom .csv')
            elif int(group['name'][4:6]) == lab_number:
                group_ids = get_group_set_groups(group['id'])
                print('Recovered group ids for output to zoom .csv')
        except: None

    df = pd.DataFrame(data=None,columns=['Pre-assign Room Name','Email Address'])

    if phys_course == 261: cap_num = 4
    else: cap_num = 5


    for grp in group_ids:
        this_name = grp['name']
        #print(this_name)
        sec = this_name.find('Section')+8
        grp_num = this_name.find('Group')+6
        section = this_name[sec:sec+3]
        #print(sec)
        if section == input_section:
            group_num = this_name[grp_num]
            for student in get('groups/{}/users'.format(grp['id'])):
                zs = student['login_id']+'@umich.edu'
                zb_series = {'Pre-assign Room Name': this_name, 'Email Address':zs}
                df = df.append(zb_series,ignore_index=True)
            if int(group_num) == cap_num:
                if not os.path.exists(zb_path):
                    os.mkdir(zb_path)
                final_path = '{}Lab {}/'.format(zb_path,lab_number)
                if not os.path.exists(final_path):
                    os.mkdir(final_path)
                print('Output {} groups to section {} lab {}'.format(group_num,section,lab_number))
                fname_zb_csv = '{}Section {} Lab {}.csv'.format(final_path,section,lab_number)
                df.to_csv(fname_zb_csv,index=False)
                df = pd.DataFrame(data=None,columns=['Pre-assign Room Name','Email Address'])
    print('Done.')


# Using CoCalc API to create projects

In [0]:
#zb_path = 'Physics {}/Zoom Groups/'.format(course)
#student_course_pid = '3890c031-24fc-478d-a267-4e454648e43d'

In [4]:
def load_user_info(fname):
    with open(fname,"r") as inf:
        user_info = yaml.load(inf,Loader=yaml.FullLoader)['api_user']
    return user_info

def load_license(fname,number):
    with open(fname,"r") as inf:
        license = yaml.load(inf,Loader=yaml.FullLoader)['license_{}'.format(number)]
    return license


In [5]:
def call_api(msg="ping", payload={}, sk = '', base_url="https://cocalc.com", max_retries=10, timeout=60):
    s = requests.Session()
    a = requests.adapters.HTTPAdapter(max_retries=max_retries)
    s.mount('https://', a)
    url = "{}/api/v1/{}".format(base_url, msg)
    auth = HTTPBasicAuth(sk,'')
    headers = {'content-type': 'application/json'}
    r = s.post(url,auth=auth,data=json.dumps(payload),headers=headers, timeout=timeout)
    try: pp.pprint(r.json())
    except: None
    assert r.status_code == requests.codes.ok,"bad status code {}".format(r.status_code)
    retval = r.json()
    return retval


In [122]:
def update_cocalc_roster(course,section,lab_number):
    zb_path = 'Physics {}/Zoom Groups/'.format(course)
    roster_path = 'Physics {}/Roster Data/'.format(course)
    if not os.path.exists(roster_path): os.mkdir(roster_path)
    roster_path = '{}Lab {}/'.format(roster_path,lab_number)
    if not os.path.exists(roster_path): os.mkdir(roster_path)
    student_course_pid = '3890c031-24fc-478d-a267-4e454648e43d'
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")
    
    payload = {'id':str(uuid4()),'project_id':student_course_pid,'path':'Course Files/Physics {}/Physics {} Section {}.course'.format(course,course,section)}
    #return call_api('read_text_file_from_project',payload,uinfo['api_key'])
    course_json = call_api('read_text_file_from_project',payload,uinfo['api_key'])
    #pp.pprint(course_json)
    final_path = '{}Lab {}/'.format(zb_path,lab_number)
    fname_zb_csv = '{}Section {} Lab {}.csv'.format(final_path,section,lab_number)
    zoom_df = pd.read_csv(fname_zb_csv)
    print(zoom_df)
    
    df = pd.DataFrame(data=None,columns=['Account ID','Email Address','Group'])
    for line in course_json['content'].split('\n'):
        #pp.pprint(line)
        row = {}
        to_json = json.loads(line)
        if to_json['table'] == 'students':
            try: 
                row['Account ID'] = to_json['account_id']
                row['Email Address'] = to_json['email_address']
            except:
                print('No account id for {}, student may not be signed up to CoCalc'.format(to_json['email_address']))
                row['Account ID'] = 'No Account'
                row['Email Address'] = to_json['email_address']
            this_group = zoom_df[zoom_df['Email Address'] == row['Email Address']]['Pre-assign Room Name'].values[0]
            row['Group'] = this_group
            df = df.append(row,ignore_index=True)
    df.to_csv('{}/Section {} Lab {}.csv'.format(roster_path,section,lab_number),index=False)

In [59]:
course = '241'
section = '010'
lab_number = '0'


zb_path = 'Physics {}/Zoom Groups/'.format(course)
roster_path = 'Physics {}/Roster Data/'.format(course)
if not os.path.exists(roster_path): os.mkdir(roster_path)
roster_path = '{}Lab {}/'.format(roster_path,lab_number)
if not os.path.exists(roster_path): os.mkdir(roster_path)
student_course_pid = '3890c031-24fc-478d-a267-4e454648e43d'
uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")

payload = {'id':str(uuid4()),'project_id':student_course_pid,'path':'Course Files/Physics {}/Physics {} Section {}.course'.format(course,course,section)}
#return call_api('read_text_file_from_project',payload,uinfo['api_key'])
course_json = call_api('read_text_file_from_project',payload,uinfo['api_key'])
#pp.pprint(course_json)
final_path = '{}Lab {}/'.format(zb_path,lab_number)
fname_zb_csv = '{}Section {} Lab {}.csv'.format(final_path,section,lab_number)
zoom_df = pd.read_csv(fname_zb_csv)
print(zoom_df)

df = pd.DataFrame(data=None,columns=['Account ID','Email Address','Group'])
for line in course_json['content'].split('\n'):
    #pp.pprint(line)
    row = {}
    to_json = json.loads(line)
    if to_json['table'] == 'students':
        try: 
            row['Account ID'] = to_json['account_id']
            row['Email Address'] = to_json['email_address']
        except:
            print('No account id for {}, student may not be signed up to CoCalc'.format(to_json['email_address']))
            row['Account ID'] = 'No Account'
            row['Email Address'] = to_json['email_address']
        this_group = zoom_df[zoom_df['Email Address'] == row['Email Address']]['Pre-assign Room Name'].values[0]
        row['Group'] = this_group
        df = df.append(row,ignore_index=True)
df.to_csv('{}/Section {} Lab {}.csv'.format(roster_path,section,lab_number),index=False)

{'content': '{"account_id":"00cc343c-61a5-4a9d-b05a-c30af8258f2e","email_address":"alkucich@umich.edu","last_email_invite":1611051113829,"project_id":"eb0607f8-feec-4b7e-8c2a-d68f5bc89f4c","student_id":"4549ea95-cd6c-4c71-91f1-7eeb6779f316","table":"students"}\n'
            '{"account_id":"21f90df6-5227-44b8-aabd-f6884bef158c","email_address":"jmate@umich.edu","last_email_invite":1611051111840,"project_id":"ea47f869-518e-4fd8-97a2-9d4a21dd4ec7","student_id":"bbef9b77-cdfa-40a5-bd4d-9b403673d175","table":"students"}\n'
            '{"account_id":"38cdf820-e2f5-47b7-bec8-96ea4618541f","email_address":"rahchowd@umich.edu","last_email_invite":1611051111782,"project_id":"df249469-ee74-40b4-a7ea-a9d5d447d96c","student_id":"82aedfaf-8631-40e2-a04d-1c590cf46996","table":"students"}\n'
            '{"account_id":"4ad8d9be-d302-4667-a00d-3269afe0fa09","email_address":"vivkim@umich.edu","last_email_invite":1611051109807,"project_id":"85b97714-f914-45fe-8156-31144905eb15","student_id":"fe0a8fef-2

IndexError: index 0 is out of bounds for axis 0 with size 0

In [57]:
course_json

{'event': 'text_file_read_from_project',
 'id': 'ee7ccd14-6a67-4a83-9702-5a22d3ad027e',
 'content': '{"account_id":"44717044-0f85-42ea-b12a-a42663c8a698","create_project":"2021-01-19T10:00:10.275Z","email_address":"benwg@umich.edu","student_id":"fd85faff-bdda-4c40-be24-396e199c4019","table":"students"}\n{"account_id":"b6db09e8-62d1-42cc-9adf-a1a1a22250f2","email_address":"dgourlay@umich.edu","student_id":"dfa037e5-7da7-4f47-b826-9c14a34ff5b6","table":"students"}\n{"create_project":"2021-01-19T10:00:09.790Z","email_address":"beaudoim@umich.edu","student_id":"25f73317-1f1e-434f-95dd-152cf6aa2765","table":"students"}\n{"create_project":"2021-01-19T10:00:10.261Z","email_address":"anushkap@umich.edu","student_id":"c3249dac-5817-478d-afe2-209e3cbb45a1","table":"students"}\n{"create_project":"2021-01-19T10:00:10.269Z","email_address":"amtorrey@umich.edu","student_id":"57e0adf9-7151-454d-a236-6444ff914bd8","table":"students"}\n{"create_project":"2021-01-19T10:00:10.272Z","email_address":"abert

In [183]:

def create_collab_groups(course,section,lab_number):
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")
    license = load_license("./Keys/License_ID.yaml",'0')
    root_project = '7c6aa327-204b-499e-a7ad-1ed208d0c2ed'
    roster_path = 'Physics {}/Roster Data/Lab {}/Section {} Lab {}.csv'.format(course,lab_number,section,lab_number)
    df = pd.read_csv(roster_path,index_col=False)
    groups = df['Group'].drop_duplicates()


    if int(section) < 50: slot = 1
    else: slot = 2
    if int(course) == 261 and int(section) == 3: slot = 2
    check_and_remove_slot_previous(course,slot)
    log_df = pd.DataFrame(data=None,columns=['project_id','account_id','email_address'])
    
    activity_log_fname = './Keys/Active_Phys{}_Slot{}.csv'.format(course,slot)
    
    gsi_df = pd.read_csv('Keys/Instructors.csv')
    gsi_df = gsi_df[gsi_df['Course'] == int(course)]
    gsi_df = gsi_df[gsi_df['Section'] == int(section)]
    gsi_email = gsi_df['Email Address'].values[0]
    gsi_acc = gsi_df['Account ID'].values[0]
    
    
    for group in groups:

        group_number = group[-1:]
        project_title = 'Section {} - Lab {} Group {}'.format(section,lab_number,group_number)
        payload = {"id":str(uuid4()),"title":project_title,"license":license['license_id'],"start":'false'}
        new_project = call_api("create_project",payload,uinfo['api_key'])
        #copy worksheet
        copied = False
        timeout = False
        copy_tries = 0
        while not copied and not timeout:
            try:
                if lab_number == 0: lab_number = '00'
                if lab_number == 1: lab_number = '01'
                path_to_worksheet = glob('Lab Worksheets & Prelabs/Physics {}/Lab *{}*/'.format(course,lab_number))[0]
                #print(path_to_worksheet)
                new_path = '/Files/'
                payload = {'id':str(uuid4()),'src_project_id':root_project,'src_path':path_to_worksheet,'target_project_id':new_project['project_id'],'target_path':new_path}
                call_api('copy_path_between_projects',payload,uinfo['api_key'])
                copied = True
            except AssertionError:
                print('***********Unable to copy worksheet files. Trying Again.*********')
                copy_tries += 1
            if copy_tries > 10: timeout = True

        for index,student in df.iterrows():
            if group == student['Group']:
                log = {}
                log['project_id'] = new_project['project_id']
                email = student['Email Address']
                log['email_address'] = email
                print('Emailing {} collaboration invite.'.format(email))
                student_cc_id = student['Account ID']
                log['account_id'] = student_cc_id
                log_df = log_df.append(log,ignore_index=True)
                if student_cc_id == 'No Account':
                #invite noncloud
                    subject = "Invitation to collaborate on Lab {} for {}".format(lab_number,course)
                    l2p = 'https://cocalc.com/projects/{}'.format(new_project['project_id'])
                    content = 'Instructor invitation for Lab {}. Please login to CoCalc and join the appropriate project with your labmates.'.format(lab_number)
                    payload = {'id':str(uuid4()),'project_id':new_project['project_id'],'to':email,'email':content,'title':project_title,'link2proj':l2p,'replyto':gsi_email,'subject':subject}
                    call_api("invite_noncloud_collaborators",payload,uinfo['api_key'])
                else:
                    subject = "Invitation to collaborate on Lab {} for {}".format(lab_number,course)
                    l2p = 'https://cocalc.com/projects/{}'.format(new_project['project_id'])
                    content = 'Instructor invitation for Lab {}. Please login to CoCalc and join the appropriate project with your labmates.'.format(lab_number)
                    payload = {'id':str(uuid4()),'project_id':new_project['project_id'],'account_id':student_cc_id,'title':project_title,'link2proj':l2p,'replyto':gsi_email,'email':content,'subject':subject}
                    call_api("invite_collaborator",payload,uinfo['api_key'])
        print('Emailing instructor {} collaboration invite.'.format(gsi_email))
        subject = "Invitation to collaborate on Lab {} for {}".format(lab_number,course)
        l2p = 'https://cocalc.com/projects/{}'.format(new_project['project_id'])
        content = 'Instructor invitation for Lab {}. Please login to CoCalc and join the appropriate project with your labmates.'.format(lab_number)
        payload = {'id':str(uuid4()),'project_id':new_project['project_id'],'account_id':gsi_acc,'title':project_title,'link2proj':l2p,'replyto':gsi_email,'email':content,'subject':subject}
        call_api("invite_collaborator",payload,uinfo['api_key'])



    log_df.to_csv(activity_log_fname)



In [46]:
def check_and_remove_slot_makeup(course,slot):
    activity_log_fname = './Keys/Active_Phys{}_Slot{}_Makeup.csv'.format(course,slot)
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")
    license = load_license("./Keys/License_ID.yaml",'0')['license_id']

    for fname in glob(activity_log_fname):
        df = pd.read_csv(fname)

        for index,info_row in df.iterrows():
            try:
                payload = {'id':str(uuid4()),'project_id':info_row['project_id'],'account_id':info_row['account_id']}
                call_api('remove_collaborator',payload,uinfo['api_key'])
            except:
                print('No account found for {}, could not remove from project.'.format(info_row['email_address']))
            project_id = info_row['project_id']
            try:
                payload = {'id':str(uuid4()),'project_id':project_id,'license_id':license}
                call_api('remove_license_from_project',payload,uinfo['api_key'])
            except: None

        os.remove(fname)


def end_makeup_session(course,slot):
    check_and_remove_slot_makeup(course,slot)


def makeup_collab_project(num_collab,course,lab_number,slot):
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")
    license = load_license("./Keys/License_ID.yaml",'0')
    root_project = '7c6aa327-204b-499e-a7ad-1ed208d0c2ed'



    check_and_remove_slot_makeup(course,slot)
    log_df = pd.DataFrame(data=None,columns=['project_id','account_id','email_address'])
    activity_log_fname = './Keys/Active_Phys{}_Slot{}_Makeup.csv'.format(course,slot)


    collab_list = pd.DataFrame(data=None,columns=['Email Address','Section','Account ID',])
    col_info = {}
    for n in range(num_collab):
        col_info = {}
        print('Input participant {} info:'.format(n+1))
        x = input('uniqname: ')
        col_info['Email'] = '{}@umich.edu'.format(x)
        x = input('Section number including zeros e.g. 034 (000 for instructors): ')
        col_info['Section'] = x
        if x == '000': roster_path = 'Keys/Instructors.csv'
        else: roster_path = 'Physics {}/Roster Data/Lab {}/Section {} Lab {}.csv'.format(course,lab_number,col_info['Section'],lab_number)
        df = pd.read_csv(roster_path,index_col=False)
        try:
            this_student = df[df['Email Address'] == col_info['Email Address']].drop_duplicates()
            col_info['Account ID'] = this_student['Account ID'].values[0]
            collab_list = collab_list.append(col_info,ignore_index=True)
        except:
            print('Student {} was not found on CoCalc roster, and must be added manually.')

    #add michelle
    #col_info['Email'] = 'mjlove@umich.edu'
    #col_info['Section'] = '000'
    #roster_path = 'Keys/Instructors.csv'
    #df = pd.read_csv(roster_path,index_col=False)
    #this_student = df[df['Email Address'] == col_info['Email']].drop_duplicates()
    #col_info['Account ID'] = this_student['Account ID'].values[0]
    #collab_list = collab_list.append(col_info,ignore_index=True)

    project_title = 'Makeup Room - Lab {}'.format(lab_number)
    payload = {"id":str(uuid4()),"title":project_title,"license":license['license_id'],"start":'false'}
    new_project = call_api("create_project",payload,uinfo['api_key'])
    #copy worksheet
    copied = False
    timeout = False
    copy_tries = 0
    while not copied and not timeout:
        try:
            if int(lab_number) == 0: path_lab_number = '00'
            elif int(lab_number) == 1: path_lab_number = '01'
            else path_lab_number = lab_number
            path_to_worksheet = glob('Lab Worksheets & Prelabs/Physics {}/Lab *{}*/'.format(course,path_lab_number))[0]
            #print(path_to_worksheet)
            new_path = '/Files/'
            payload = {'id':str(uuid4()),'src_project_id':root_project,'src_path':path_to_worksheet,'target_project_id':new_project['project_id'],'target_path':new_path}
            call_api('copy_path_between_projects',payload,uinfo['api_key'])
            copied = True
        except AssertionError:
            print('***********Unable to copy worksheet files. Trying Again.*********')
            copy_tries += 1
        if copy_tries > 10: timeout = True
        
            

    for index,student in collab_list.iterrows():
        log = {}
        log['project_id'] = new_project['project_id']
        email = student['Email Address']
        log['email_address'] = email
        print('Emailing {} collaboration invite.'.format(email))
        student_cc_id = student['Account ID']
        log['account_id'] = student_cc_id
        log_df = log_df.append(log,ignore_index=True)
        if student_cc_id == 'No Account':
        #invite noncloud
            subject = "Invitation to collaborate on Lab {} for {}".format(lab_number,course)
            l2p = 'https://cocalc.com/projects/{}'.format(new_project['project_id'])
            content = 'Instructor invitation for Lab {}. Please login to CoCalc and join the appropriate project with your labmates.'.format(lab_number)
            payload = {'id':str(uuid4()),'project_id':new_project['project_id'],'to':email,'email':content,'title':project_title,'link2proj':l2p,'replyto':'mjlove@umich.edu','subject':subject}
            call_api("invite_noncloud_collaborators",payload,uinfo['api_key'])
        else:
            subject = "Invitation to collaborate on Lab {} for {}".format(lab_number,course)
            l2p = 'https://cocalc.com/projects/{}'.format(new_project['project_id'])
            content = 'Instructor invitation for Lab {}. Please login to CoCalc and join the appropriate project with your labmates.'.format(lab_number)
            payload = {'id':str(uuid4()),'project_id':new_project['project_id'],'account_id':student_cc_id,'title':project_title,'link2proj':l2p,'replyto':'mjlove@umich.edu','email':content,'subject':subject}
            call_api("invite_collaborator",payload,uinfo['api_key'])

    log_df.to_csv(activity_log_fname)

In [26]:
def update_gsi_roster():
    student_course_pid = '3890c031-24fc-478d-a267-4e454648e43d'
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")

    payload = {'id':str(uuid4()),'project_id':student_course_pid,'path':'Instructors.course'}
    course_json = call_api('read_text_file_from_project',payload,uinfo['api_key'])

    df = pd.DataFrame(data=None,columns=['Account ID','Email Address','Sections','Course'])
    for line in course_json['content'].split('\n'):
        row = {}
        to_json = json.loads(line)
        if to_json['table'] == 'students':
            try: 
                row['Account ID'] = to_json['account_id']
                row['Email Address'] = to_json['email_address']
            except:
                print('No account id for {}, student may not be signed up to CoCalc'.format(to_json['email_address']))
                row['Account ID'] = 'No Account'
                row['Email Address'] = to_json['email_address']
            print('Current Instructor: {}'.format(row['Email Address']))
            row['Course'] = input('Input instructor course number e.g. 141')
            x = input('Input instructor sections, separated by a comma and space:')
            sects = x.split(', ')
            for section in sects:
                row['Section'] = section
                df = df.append(row,ignore_index=True)
    df.to_csv('./Keys/Instructors.csv',index=False)

In [6]:
def check_gsi_accs():
    student_course_pid = '3890c031-24fc-478d-a267-4e454648e43d'
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")

    payload = {'id':str(uuid4()),'project_id':student_course_pid,'path':'Instructors.course'}
    course_json = call_api('read_text_file_from_project',payload,uinfo['api_key'])

In [45]:
from glob import glob
import os


def check_and_remove_slot_previous(course,slot):
    activity_log_fname = './Keys/Active_Phys{}_Slot{}.csv'.format(course,slot)
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")
    license = load_license("./Keys/License_ID.yaml",'0')['license_id']

    for fname in glob(activity_log_fname):
        df = pd.read_csv('fname')

        for index,info_row in df.iterrows():
            try:
                payload = {'id':str(uuid4()),'project_id':info_row['project_id'],'account_id':info_row['account_id']}
                call_api('remove_collaborator',payload,uinfo['api_key'])
            except:
                print('No account found for {}, could not remove from project.'.format(info_row['email_address']))
            project_id = info_row['project_id']

        payload = {'id':str(uuid4()),'project_id':project_id,'license_id':license}
        call_api('remove_license_from_project',payload,uinfo['api_key'])
        os.remove(fname)


def end_section_session(course,section):
    if int(section) < 50: slot = 1
    else: slot = 2
    if int(course) == 261 and int(section) == 3: slot = 2
    check_and_remove_slot_previous(course,slot)

In [12]:
stu_sect = get_sections(course_id,include=['students'])

In [17]:
for section in stu_sect:
    print(section['name'])
    for student in section['students']:
        print(student['name'])

PHYSICS 241 Section 003
Manisha Sri Solipuram
Jacob Charles Florian
Medha Kumar
David Georgi
Brian Delgado
Bradley Lin Wang
Aryanna Marie Thompson
Roberts Maris Kalnins
Adelaide Marie Barcalow
Marion Chunmei Ni
Sophie Zucker
Cole Edgar Whidden
Vincent Christopher Larsson
Kaleigh Miller
Rohun A Athalye
Jonathan Richardson
Anshul Friedman-Jha
Lynette K Lee
Nicole Agabon Kuchta
Sahil Anil Patel
PHYSICS 241 Section 010
Tao Wang
Tianer Zhou
Zaynab Omar Elkolaly
Kaden O'Loughlin
Dhiya Krupashankar
Andrew Brian Chen
John Kenneth Collison
Vivian Sang-Me Kim
Prashant Saxena
Thomas Edward Dailey
Rahul Chowdhury
David Martin Mutone
Zachary Joseph Cary
Sheng Bai
Matthew Shantha Weerakoon
Alexandra A Kucich
Jayesh Suchit Mate
Dahlia Marie Evans
Adam Julian Saifuddin
Kathleen Anne Bray
PHYSICS 241 Section 011
Nicholas Logan Tiberia
Katharine Severson
Alexandra Maria Stavrakos
Scott Ai
Max David Crandell
Eva Mason
Trenton Earl McLean
Kimberly Lynnae Meagher
Benjamin Samuel Routhier
Aanand Hemang Shah

In [15]:
def update_cocalc_roster(course,section,lab_number):
    roster_path = 'Physics {}/Roster Data/'.format(course)
    if not os.path.exists(roster_path): os.mkdir(roster_path)
    roster_path = '{}Lab {}/'.format(roster_path,lab_number)
    if not os.path.exists(roster_path): os.mkdir(roster_path)
    student_course_pid = '3890c031-24fc-478d-a267-4e454648e43d'
    uinfo = load_user_info("./Keys/LGSI_API_KEY.yaml")
    
    payload = {'id':str(uuid4()),'project_id':student_course_pid,'path':'Course Files/Physics {}/Physics {} Section {}.course'.format(course,course,section)}
    #return call_api('read_text_file_from_project',payload,uinfo['api_key'])
    course_json = call_api('read_text_file_from_project',payload,uinfo['api_key'])
    #pp.pprint(course_json)
    final_path = '{}Lab {}/'.format(zb_path,lab_number)
    fname_zb_csv = '{}Section {} Lab {}.csv'.format(final_path,section,lab_number)
    zoom_df = pd.read_csv(fname_zb_csv)
    print(zoom_df)
    
    df = pd.DataFrame(data=None,columns=['Account ID','Email Address','Group'])
    for line in course_json['content'].split('\n'):
        #pp.pprint(line)
        row = {}
        to_json = json.loads(line)
        if to_json['table'] == 'students':
            try: 
                row['Account ID'] = to_json['account_id']
                row['Email Address'] = to_json['email_address']
            except:
                print('No account id for {}, student may not be signed up to CoCalc'.format(to_json['email_address']))
                row['Account ID'] = 'No Account'
                row['Email Address'] = to_json['email_address']
            this_group = zoom_df[zoom_df['Email Address'] == row['Email Address']]['Pre-assign Room Name'].values[0]
            row['Group'] = this_group
            df = df.append(row,ignore_index=True)
    df.to_csv('{}/Section {} Lab {}.csv'.format(roster_path,section,lab_number),index=False)

{'id': 464839,
 'course_id': 436130,
 'name': 'PHYSICS 241 Section 003',
 'start_at': None,
 'end_at': None,
 'created_at': '2020-10-03T01:00:26Z',
 'restrict_enrollments_to_section_dates': False,
 'nonxlist_course_id': None,
 'students': [{'id': 452872,
   'name': 'Manisha Sri Solipuram',
   'created_at': '2018-03-29T21:42:13-04:00',
   'sortable_name': 'Solipuram, Manisha Sri',
   'short_name': 'Manisha Sri Solipuram',
   'root_account': 'umich.instructure.com',
   'login_id': 'manishas'},
  {'id': 415029,
   'name': 'Jacob Charles Florian',
   'created_at': '2017-04-17T07:50:14-04:00',
   'sortable_name': 'Florian, Jacob Charles',
   'short_name': 'Jacob Charles Florian',
   'root_account': 'umich.instructure.com',
   'login_id': 'florianj'},
  {'id': 415402,
   'name': 'Medha Kumar',
   'created_at': '2017-04-18T06:33:10-04:00',
   'sortable_name': 'Kumar, Medha',
   'short_name': 'Medha Kumar',
   'root_account': 'umich.instructure.com',
   'login_id': 'medhak'},
  {'id': 531697,
